In [1]:
# Applied Energy

In [2]:
idNo = '271429'

In [3]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd
import json
import csv

In [4]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [6]:
# searchTerm1 = ['Building Information Modelling']
# searchTerm2 = ['Green Buildings']

# searchTerm1 = ['energy']
# searchTerm2 = ['green']

tc = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "%20"

    fileIndex = text2.rfind("%20")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "%20"

    fileIndex = text1.rfind("%20")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

['Green%20Building', 'Green%20Buildings', 'Sustainable%20building', 'Sustainable%20buildings', 'Environmentally%20friendly%20design', 'Sustainability', 'Sustainable%20development', 'Energy', 'Water', 'Waste', 'Thermal%20comfort', 'Electricity', 'Solar', 'Wind%20power', 'Green%20infrastructure', 'Life%20cycle%20assessment']
['BIM', 'Building%20Information%20Modelling', 'Building%20information%20model', '6D%20BIM', 'Digital%20construction', 'Smart%20construction', 'Digital%20engineering']


In [7]:
def getData(link):
    browser = webdriver.Safari()
    browser.get(link)
    respData = browser.page_source
    browser.quit()
    parsed_html = BeautifulSoup(respData)
    
    title = parsed_html.find('span',{'class':'title-text'}).text
    print("Title: ",title)
    df.loc[row,'url'] = link
    df.loc[row,'Title'] = title
    
    absResults = parsed_html.find('div',{'class':'abstract author'})
    
    if absResults is not None:
        try:            
            df.loc[row,'section0'] = absResults.find('h2').text
            df.loc[row,'sectionData0'] = absResults.find('div').text
        except:
            df.loc[row,'section0'] = "None"
            df.loc[row,'sectionData0'] = "None"
    else:
        df.loc[row,'section0'] = "None"
        df.loc[row,'sectionData0'] = "None"

        
    sections = parsed_html.find('div',{'class':'Body u-font-serif'})
    if sections is not None:        
        sections = sections.find_next('div')
        count = 1 
        for section in sections:
            if section is not None: 
                sectionNo = 'section' + str(count)
                sectionData = 'sectionData' + str(count)
                try:
                    df.loc[row,sectionNo] = section.find('h2',{'class':'u-h3 u-margin-l-top u-margin-xs-bottom'}).text
                except:
                    df.loc[row,sectionNo] = "Some issue"
                    
                data = ''
                for para in section.findAll('p'):
                    data = data + para.text
                df.loc[row,sectionData] = data
            else:
                df.loc[row,sectionNo] = "None"
            count = count + 1        

In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [9]:
# https://www.sciencedirect.com/search/api?qs=%22BIM%22%20%2B%20%22Green%20Building%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com

baseUrl = 'https://www.sciencedirect.com/search/api?qs='

querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        sq = 'https://www.sciencedirect.com/search?qs=%22' + item1 + '%22%20%2B%20%22' + item2 + '%22'
        print("base query: ",sq)
        query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&lastSelectedFacet=publicationTitles&publicationTitles=" + idNo + "&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
        
        # Call webdriver         
        browser = webdriver.Safari()
        browser.get(query)
        respData = browser.page_source
        browser.quit()

        # Clear html         
        respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
        respData = respData.replace('</pre></body></html>',"")
        
        py_dic = json.loads(respData)
        
        # Finding the number of pages in the selected journals.         
        for key in py_dic:
            if key == 'resultsFound':
                pageResults = py_dic[key]
        print("pageResults: ",pageResults)                
                                
        if pageResults > 25:
            page = pageResults // 25
            lastPage = pageResults % 25
            if lastPage > 0:
                page = page + 1
        else:
            page = 1
        print("page: ",page)

        #         for p in range(2):        
        for p in range(page):
            print("In page: ",p)
            if p == 0:
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&lastSelectedFacet=publicationTitles&publicationTitles="+idNo+"&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
            else:
                offset = '&offset='+str(p*25)
                query = baseUrl + item1 +"%22%20%2B%20%22"+ item2+"%22&lastSelectedFacet=publicationTitles&publicationTitles="+idNo+offset+"&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com"
                
            print("search query: ",query)  
                
            # Call webdriver         
            browser = webdriver.Safari()
            browser.get(query)
            respData = browser.page_source
            browser.quit()
            # Load Json Data 

            respData = respData.replace('<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">',"")
            respData = respData.replace('</pre></body></html>',"")

            py_dic = json.loads(respData)                
            # Quit the browser 
            links = []
            for key in py_dic:
                if key == 'searchResults':
                    for searchR in py_dic[key]:
                        for search in searchR:
                            if search == 'pdf':
                                for linkR in searchR[search]:
                                    if linkR == 'getAccessLink':
                                        baseLink = 'https://www.sciencedirect.com/'
                                        link = baseLink +searchR[search][linkR]
                                        print('Reading result: ',row)
                                        df.loc[row,'searchTerm1'] = item1
                                        df.loc[row,'searchTerm2'] = item2
                                        getData(link)                                    
                                        row = row + 1        

base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Green%20Building%22
pageResults:  10
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=BIM%22%20%2B%20%22Green%20Building%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  0
Title:  Evolutionary double attention-based long short-term memory model for building energy prediction: Case study of a green building
Reading result:  1
Title:  Integrated optimization with building information modeling and life cycle assessment for generating energy efficient buildings
Reading result:  2
Title:  Building information modelling based building energy modelling: A review
Reading result:  3
Title:  A framework for producing gbXML building geometry from P

Title:  Characterization of cooling loads in the wine industry and novel seasonal indicator for reliable assessment of energy saving through retrofit of chillers
Reading result:  40
Title:  A simulation and optimisation methodology for choosing energy efficiency measures in non-residential buildings
Reading result:  41
Title:  Optimisation of biogas generation from brown seaweed residues: Compositional and geographical parameters affecting the viability of a biorefinery concept
Reading result:  42
Title:  Evolutionary double attention-based long short-term memory model for building energy prediction: Case study of a green building
Reading result:  43
Title:  Automated metamodel generation for Design Space Exploration and decision-making – A novel method supporting performance-oriented building design and retrofitting
Reading result:  44
Title:  Computational tools for design, analysis, and management of residential energy systems
Reading result:  45
Title:  Ceramic panels versus alumin

Title:  Automated metamodel generation for Design Space Exploration and decision-making – A novel method supporting performance-oriented building design and retrofitting
Reading result:  88
Title:  Brick : Metadata schema for portable smart building applications
Reading result:  89
Title:  Component-based machine learning for performance prediction in building design
Reading result:  90
Title:  Optimisation of biogas generation from brown seaweed residues: Compositional and geographical parameters affecting the viability of a biorefinery concept
Reading result:  91
Title:  High-resolution, parametric BIPV and electrical systems modeling and design
Reading result:  92
Title:  Options for residential building services design using fuel cell based micro-CHP and the potential for heat integration
Reading result:  93
Title:  Approximate model predictive building control via machine learning
base query:  https://www.sciencedirect.com/search?qs=%22BIM%22%20%2B%20%22Water%22
pageResults:  25
p

Title:  ExRET-Opt: An automated exergy/exergoeconomic simulation framework for building energy retrofit analysis and design optimisation
Reading result:  136
Title:  Linking energy-cyber-physical systems with occupancy prediction and interpretation through WiFi probe-based ensemble classification
Reading result:  137
Title:  Automated metamodel generation for Design Space Exploration and decision-making – A novel method supporting performance-oriented building design and retrofitting
Reading result:  138
Title:  Computational tools for design, analysis, and management of residential energy systems
Reading result:  139
Title:  A daily baseline model based on transfer functions for the verification of energy saving. A case study of the administration room at the Palacio de la Madraza, Granada
Reading result:  140
Title:  Building occupant transient agent-based model – Movement module
Reading result:  141
Title:  Approximate model predictive building control via machine learning
Reading r

Title:  Optimal design of timber-glass upgrade modules for vertical building extension from the viewpoints of energy efficiency and visual comfort
Reading result:  186
Title:  Data-driven Urban Energy Simulation (DUE-S): A framework for integrating engineering simulation and machine learning methods in a multi-scale urban energy modeling workflow
Reading result:  187
Title:  Feature assessment frameworks to evaluate reduced-order grey-box building energy models
Reading result:  188
Title:  High-resolution, parametric BIPV and electrical systems modeling and design
Reading result:  189
Title:  Ceramic panels versus aluminium in buildings: Energy consumption and environmental impact assessment with a new methodology
Reading result:  190
Title:  Energy efficiency and lighting design in courtyards and atriums: A predictive method for daylight factors
Reading result:  191
Title:  Characterization of cooling loads in the wine industry and novel seasonal indicator for reliable assessment of e

...

Reading result:  630
Title:  Options for residential building services design using fuel cell based micro-CHP and the potential for heat integration
Reading result:  631
Title:  A modelling study for the integration of a PEMFC micro-CHP in domestic building services design
Reading result:  632
Title:  ExRET-Opt: An automated exergy/exergoeconomic simulation framework for building energy retrofit analysis and design optimisation
base query:  https://www.sciencedirect.com/search?qs=%22Building%20information%20model%22%20%2B%20%22Wind%20power%22
pageResults:  3
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=Building%20information%20model%22%20%2B%20%22Wind%20power%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:

Reading result:  647
Title:  Building information modelling based building energy modelling: A review
base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Water%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=6D%20BIM%22%20%2B%20%22Water%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
Reading result:  648
Title:  Building information modelling based building energy modelling: A review
base query:  https://www.sciencedirect.com/search?qs=%226D%20BIM%22%20%2B%20%22Waste%22
pageResults:  1
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=6D%20BIM%22%20%2B%20%22Waste%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252B

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Energy%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=Digital%20construction%22%20%2B%20%22Energy%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20construction%22%20%2B%20%22Water%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=Digital%20construction%22%20%2B%20%22Water%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflU

Reading result:  654
Title:  Evolutionary double attention-based long short-term memory model for building energy prediction: Case study of a green building
base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Sustainable%20development%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=Smart%20construction%22%20%2B%20%22Sustainable%20development%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Smart%20construction%22%20%2B%20%22Energy%22
pageResults:  3
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=Smart%20construction%22%20%2B%20%22Energy%22&lastSelectedFacet=publicationTitles&public

base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Sustainable%20buildings%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=Digital%20engineering%22%20%2B%20%22Sustainable%20buildings%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk560OvG01SG7BOVWTh3dBnVFIyskTP8RshDmFC8bOh%252FESLEnz71aglF%252FsYdi25FYKLtovKdDbfVcomCzYflUlyb3MA%253D%253D&hostname=www.sciencedirect.com
base query:  https://www.sciencedirect.com/search?qs=%22Digital%20engineering%22%20%2B%20%22Environmentally%20friendly%20design%22
pageResults:  0
page:  1
In page:  0
search query:  https://www.sciencedirect.com/search/api?qs=Digital%20engineering%22%20%2B%20%22Environmentally%20friendly%20design%22&lastSelectedFacet=publicationTitles&publicationTitles=271429&t=7EN%252Bo5sQQ6zhZ%252BAoTco%252BCCKhD05wwee9t4ehN2046v1mpH7xwea%252FkQji4enNk5

In [12]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19,journalName
0,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Evolutionary double attention-based long short...,Abstract,The prediction of building energy consumption ...,1. Introduction,Building energy consumption has become a signi...,2. Literature review,A deviation of more than 30% is found between ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
1,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Integrated optimization with building informat...,Abstract,Energy consumption in buildings is a very impo...,1. Introduction,The construction industry is known for its sig...,2. Materials and methods,The novelty of this work is to enhance the eff...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
2,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
3,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,A framework for producing gbXML building geome...,Abstract,The industrial sector accounts for 17% of end-...,1. Introduction,In 2012 the end-use energy by industry account...,2. Methodology,This research presents a conceptual framework ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
4,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,A study on building performance analysis for e...,Abstract,Due to the strengthening of regulations and co...,1. Introduction,The industrial sector is one of the largest en...,2. Method,For the analysis and simulation of building pe...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,Digital%20engineering,Sustainability,https://www.sciencedirect.com//science/article...,Screening and techno-economic assessment of bi...,Abstract,Biomass-based power generation combined with C...,1. Introduction,The International Energy Agency (IEA) has warn...,2. Overview of biopower CCS,From the perspective of deployment of Biopower...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
664,Digital%20engineering,Energy,https://www.sciencedirect.com//science/article...,Screening and techno-economic assessment of bi...,Abstract,Biomass-based power generation combined with C...,1. Introduction,The International Energy Agency (IEA) has warn...,2. Overview of biopower CCS,From the perspective of deployment of Biopower...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
665,Digital%20engineering,Waste,https://www.sciencedirect.com//science/article...,Screening and techno-economic assessment of bi...,Abstract,Biomass-based power generation combined with C...,1. Introduction,The International Energy Agency (IEA) has warn...,2. Overview of biopower CCS,From the perspective of deployment of Biopower...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429
666,Digital%20engineering,Electricity,https://www.sciencedirect.com//science/article...,Screening and techno-economic assessment of bi...,Abstract,Biomass-based power generation combined with C...,1. Introduction,The International Energy Agency (IEA) has warn...,2. Overview of biopower CCS,From the perspective of deployment of Biopower...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271429


In [14]:
df1 = df
df['journalName'] = str(idNo)
fileSave = 'kgRawData/SD/sd-kg'+idNo+'.csv'
df.to_csv(fileSave)

In [1]:
idNo = '271429'

import pandas as pd
# filePath = 
data = pd.read_csv('kgRawData/SD/sd-kg' + idNo + '.csv')
data['presence'] = False

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

validEntries = []

for i in range(data.shape[0]):    
    for st1 in searchTerm1:
        for st2 in searchTerm2:
            if (st1 in data['sectionData0'][i]) and (st2 in data['sectionData0'][i]):
                data['presence'][i] = True
                validEntries.append(i)

uniqueValidEntries = list(set(validEntries))

uniqueDf = pd.DataFrame()
uniqueDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

uniqueDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
uniqueDf = uniqueDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    uniqueDf[section] = ''
    uniqueDf[sectionData] = ''
    
for entry in range(len(uniqueValidEntries)):
    uniqueDf.loc[entry] = data.loc[uniqueValidEntries[entry]]    
    

uniqueDf.to_csv('kgRawData/SD/cleanSd-kg'+ idNo + '.csv')

print(len(uniqueDf))

<ipython-input-1-4a2d898bd994>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['presence'][i] = True


32


In [2]:
uniqueDf

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Building%20Information%20Modelling,Solar,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6D%20BIM,Green%20Building,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Thermal%20comfort,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6D%20BIM,Green%20Buildings,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,Some issue,NaN,Some issue,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6D%20BIM,Sustainability,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6D%20BIM,Energy,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Building%20information%20model,Water,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Building%20Information%20Modelling,Energy,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This section introduces the main stages of the...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,6D%20BIM,Water,https://www.sciencedirect.com//science/article...,Building information modelling based building ...,Abstract,The emerging of building information modelling...,1. Introduction,"In the last decades, the energy consumption of...",2. Building Information Modelling and building...,This s